## The Situation

The IT department supplies the data in a SQlite database which contains 3 tables due to data protection regulations:

-   `insurance_feats`: Table of attributes (features) of the beneficiaries
-   `id_table`: Connection between the attributes and the target variable
-   `insurance_targets`: Target variable with corresponding ID

**You are asked to read in the data and provide one clean table**

## Data Import

First, connect to database via dplyr:

In [ ]:
library(dplyr)
db = src_sqlite("data/insurance.sqlite3")

In [ ]:
id_table = tbl(db, "id_table") %>% data.frame()
insurance_feats = tbl(db, "insurance_feats") %>% data.frame()
insurance_targets = tbl(db, "insurance_targets") %>% data.frame()

Then, we can merge the dataframes.

In [ ]:
insurance = insurance_feats %>%
  left_join(id_table, by = c("id" = "id1")) %>%
  left_join(insurance_targets, by = c("id2" = "id"))
insurance_raw = insurance
head(insurance_raw)

Finally, we remove the IDs.

In [ ]:
insurance = insurance %>% select(-id, -id2)
head(insurance)

## Data Cleaning + EDA

After creating the full dataset, a colleague shows you some strange results:

In [ ]:
lm(charges ~ bmi, data = insurance)

**Take a closer look at the data and check it for inconsistencies**

We make some plots of the dataset:

In [ ]:
summary(insurance[, c("age", "sex", "bmi", "children", "smoker", "region")])

The plots show indicate some problems:

- `bmi` contains outliers $\rightarrow$ how can we detect them?
- We have missing values in `age` which are coded as `-999` $\rightarrow$ transformation to real missings
- Real missings in `sex` and `bmi` $\rightarrow$ what should we do with missings?


Lets start with recoding the missing values.

In [ ]:
insurance$age[insurance$age == -999] = NA

Then we try to detect the outliers:

In [ ]:
library(ggplot2)
ggplot(insurance, aes(x = "", y = bmi)) + geom_boxplot() 

We should not use `mean` or `sd`, because these functions are very sensitive to outliers:

In [ ]:
mean(insurance$bmi)
median(insurance$bmi)
sd(insurance$bmi)
mad(insurance$bmi)

Instead, we use the median and 5 times the mean absolute deviation as a outer bound for outliers:

In [ ]:
med = median(insurance$bmi)
s = mad(insurance$bmi)
outlier = insurance$bmi > med + 5 * s | insurance$bmi < med - 5 * s
insurance$bmi[outlier] = NA

### Cleaned Dataset

Let's now have a look at the cleaned dataset:



### Effect of Outliers

To see how outliers might effect the modeling process, we train to linear models on the original and the cleaned data respectively:

In [ ]:
mod_orig = lm(charges ~ bmi, data = insurance_raw)
mod = lm(charges ~ bmi, data = insurance)

Let's visualize the effect:

In [ ]:
mod_orig = lm(charges ~ bmi, data = insurance_raw)
mod = lm(charges ~ bmi, data = insurance)

df_abline = data.frame(intercept = c(mod$coefficients[1], mod_orig$coefficients[1]), slope = c(mod$coefficients[2], mod_orig$coefficients[2]),
  Data = c("After Processing", "Before Processing"))

ggplot() +
  geom_point(data = insurance, mapping = aes(x = bmi, y = charges), size = 2, shape = 16, alpha = 0.5) +
  geom_abline(data = df_abline, mapping = aes(intercept = intercept, slope = slope, color = Data)) +
  scale_x_continuous(limits = c(15, 60))

# Exploratory Data Analysis

After cleaning the dataset you want to do a first exploratory analysis.
To get a better idea of the data, you want to illustrate simple dependencies between the interesting variable `charges` and the features.

**Therefore, visualize univariate and multivariate dependencies with focus on `charges`**

###  Charges vs Age

In [ ]:
ggplot(insurance, aes(x = age, y = charges)) +
  geom_point(alpha = 0.4)

### Charges vs Gender

In [ ]:
ggplot(insurance, aes(x = sex, y = charges)) + geom_boxplot()

### Charges vs BMI

In [ ]:
ggplot(insurance, aes(x = bmi, y = charges)) +
  geom_point(alpha = 0.4)

### Charges vs Children

In [ ]:
ggplot(insurance, aes(x = as.factor(children), y = charges)) +
  geom_boxplot(alpha = 0.4)

### Charges vs Smoker

In [ ]:
ggplot(insurance, aes(x = smoker, y = charges)) + geom_boxplot()

### Charges vs Region

In [ ]:
ggplot(insurance, aes(x = region, y = charges)) + geom_boxplot()